# 000 Forecasting Bot

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IFP import IFP, list_questions

In [3]:
ifps = list_questions()['results']

In [4]:
today_ids = list(sorted([x['id'] for x in ifps])) if ifps else [27238, 27239, 27240, 27241]
today_ids

[27876, 27877, 27885, 27886, 27887]

In [5]:
ifps = {id: IFP(id) for id in today_ids}

In [6]:
from Forecaster import Forecaster

In [7]:
fcst = Forecaster(ifps)

In [15]:
fcst.fit()

MODEL DOMAIN
27876 Will Russia control Pokrovsk Before October 1, 2024?
Military conflict
27877 Will there be a frontier open-source AI model on October 1, 2024?
AI performance
27885 Will Super Micro Computer, Inc., file its 10-K annual report with the SEC before October 1, 2024?
Market price
27886 Will Taylor Swift win 4 or more awards at the 2024 MTV Video Music Awards?
Entertainment
27887 Will Baby Reindeer win the 2024 Primetime Emmy Award for Outstanding Limited or Anthology Series?
Entertainment
CORRELATOR
NEWS
RATES
FORECASTING Frontier open-source AI model
27877 95 The available information strongly supports that Meta's Llama 3.1 405B model is already considered a "frontier-level open-source AI model." Given Meta's release and the significant adoption and positive reception of their Llama models, it is highly probable that this model or a close variant will be recognized as a top-tier open-source AI model by October 1, 2024. This aligns with statements from Meta's CEO and indus

In [16]:
for ifp in ifps.values():
    try:
        print(ifp.id, ifp.forecast)
    except:
        print(ifp.id, 'none')

27876 70
27877 95
27885 15
27886 85
27887 35


In [17]:
fcst.report()

27876 Will Russia control Pokrovsk Before October 1, 2024?
Forecast 70
Rationale The situation around Pokrovsk is described as critical and under intense pressure from Russian advances. Given the strategic importance of Pokrovsk and the reported heavy battles and advancements, it is highly plausible that Russia might achieve control by the end of September. However, various reports suggest heavy resistance from Ukrainian forces, which could delay or complicate Russian plans. Based on the available information and the deadline of October 1, I assess a 70% probability of Russia controlling Pokrovsk by that date. 

27877 Will there be a frontier open-source AI model on October 1, 2024?
Forecast 95
Rationale The available information strongly supports that Meta's Llama 3.1 405B model is already considered a "frontier-level open-source AI model." Given Meta's release and the significant adoption and positive reception of their Llama models, it is highly probable that this model or a close v

In [18]:
fcst.upload()

Prediction posted for  27876
Comment posted for  27876
Prediction posted for  27877
Comment posted for  27877
Prediction posted for  27885
Comment posted for  27885
Prediction posted for  27886
Comment posted for  27886
Prediction posted for  27887
Comment posted for  27887
